In [1]:
import os, json, cv2, numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gradio as gr


In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data", "dataset")
MODEL_DIR = os.path.join(BASE_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)
MODEL_PATH = os.path.join(MODEL_DIR, "deepfake_detector_keras.h5")
LABELS_PATH = os.path.join(MODEL_DIR, "labels.json")

IMG_SIZE = (128,128)
BATCH = 32
EPOCHS = 5


In [3]:
def make_model(input_shape=(128,128,3)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [4]:
if not os.path.exists(DATA_DIR):
    raise SystemExit(f"Dataset folder not found: {DATA_DIR}. Put images under real/ and fake/ subfolders.")

train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH, class_mode="binary", subset="training"
)

val_gen = train_datagen.flow_from_directory(
    DATA_DIR, target_size=IMG_SIZE, batch_size=BATCH, class_mode="binary", subset="validation"
)

model = make_model((IMG_SIZE[0], IMG_SIZE[1], 3))
model.summary()

ckpt = callbacks.ModelCheckpoint(MODEL_PATH, monitor="val_accuracy", save_best_only=True, mode="max")
es = callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)

history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[ckpt, es])
model.save(MODEL_PATH)
print("Model saved at:", MODEL_PATH)


SystemExit: Dataset folder not found: d:\Mtech\Research paper\DeepFake Detection\Minor Version\scripts\data\dataset. Put images under real/ and fake/ subfolders.

C:\Users\LOQ\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image

model = load_model(MODEL_PATH)

def predict(img):
    img = np.array(img)
    img = cv2.resize(img, IMG_SIZE) / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)[0][0]
    label = "Real" if pred > 0.5 else "Fake"
    confidence = pred if label == "Real" else 1 - pred
    return {
        "Real": float(confidence) if label == "Real" else float(1 - confidence),
        "Fake": float(confidence) if label == "Fake" else float(1 - confidence)
    }


In [ ]:
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Deepfake Detector (Minor Project)",
    description="Upload an image or use webcam to classify as Real or Fake."
)
demo.launch()
